In [1]:
import psycopg2
from config import local_host, local_name, local_port, local_user
from creds import local_db_password
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import random
warnings.simplefilter('ignore')
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [2]:
fixture = 1035370

In [3]:
stats_sql = f"""
select
	player_id,
	current_team_id,
	minutes,
	position,
	coalesce(shots_total * 1.0 / nullif(minutes, 0), 0) as shots_per_minute,
	coalesce(shots_on * 1.0 / nullif(shots_total, 0), 0) as shots_on_rate,
	coalesce(goals_total * 1.0 / nullif(shots_on, 0), 0) as goals_per_shot_on,
	coalesce(dribbles_attempted *1.0 / nullif(minutes, 0), 0) as dribbles_per_minute,
	coalesce(dribbles_success *1.0 / nullif(dribbles_attempted, 0), 0) as dribble_success_rate,
	coalesce(passes_total * 1.0 / nullif(minutes, 0), 0) as passes_per_minute,
	coalesce(passes_accuracy *1.0 / nullif(passes_total, 0), 0) as pass_accuracy,
	coalesce(passes_key * 1.0 / nullif(passes_accuracy, 0), 0) as key_pass_percentage,
	coalesce(assists *1.0 / nullif(passes_key, 0), 0) as assist_rate_per_key_pass,
	coalesce((tackles_total + interceptions + blocks)*1.0 / nullif(minutes, 0), 0) as defensive_moves_per_minute,
	coalesce(foul_committed * 1.0 / nullif(minutes, 0), 0) as fouls_committed_per_minute,
	coalesce(foul_drawn * 1.0 / nullif(minutes, 0), 0) as fouls_drawn_per_minute,
	coalesce(cards_yellow *1.0 / nullif(minutes, 0), 0) as cards_yellow_per_minute,
	coalesce(cards_red *1.0 / nullif(minutes, 0), 0) as cards_red_per_minute,
	coalesce(penalty_committed * 1.0 / nullif(minutes, 0), 0) as penalties_committed_per_minute,
	coalesce(saves *1.00 / nullif(goals_conceded + saves, 0), 0) as save_percentage
from ffl_prod.player_summaries
where player_id in (select player_id from ffl.fixtures_tactics where fixture_id = {fixture})
"""

fixture_info = f"""
select team_home, team_away from ffl.fixtures where id = {fixture};
"""
cur.execute(fixture_info)
teams_list = []
for row in cur.fetchall():
    team_home = row[0]
    team_away = row[1]

In [4]:
cur.execute(fixture_info)
teams_list = []
for row in cur.fetchall():
    print(row)
    team_home = row[0]
    team_away = row[1]


(65, 33)


In [5]:
df = pd.read_sql(stats_sql, conn)

In [6]:
df.head()

,player_id,current_team_id,minutes,position,shots_per_minute,shots_on_rate,goals_per_shot_on,dribbles_per_minute,dribble_success_rate,passes_per_minute,pass_accuracy,key_pass_percentage,assist_rate_per_key_pass,defensive_moves_per_minute,fouls_committed_per_minute,fouls_drawn_per_minute,cards_yellow_per_minute,cards_red_per_minute,penalties_committed_per_minute,save_percentage
0,174,33,1160,Midfielder,0.006897,0.500000,0.250000,0.005172,0.500000,0.650000,0.840849,0.042587,0.074074,0.024138,0.006034,0.002586,0.000862,0.000000,0.0,0.000000
1,526,33,3420,Goalkeeper,0.000000,0.000000,0.000000,0.000292,1.000000,0.401462,0.703569,0.003106,0.000000,0.000585,0.000292,0.001170,0.001462,0.000000,0.0,0.719807
2,742,33,1377,Defender,0.004357,0.333333,0.500000,0.001452,0.500000,0.562092,0.892765,0.001447,0.000000,0.037037,0.005810,0.002179,0.001452,0.000000,0.0,0.000000
3,886,33,3174,Defender,0.006301,0.350000,0.285714,0.015438,0.632653,0.470384,0.837240,0.030400,0.078947,0.042218,0.014178,0.004411,0.001890,0.000315,0.0,0.000000
4,909,33,2291,Attacker,0.017023,0.538462,0.333333,0.044086,0.475248,0.298560,0.792398,0.046125,0.080000,0.010476,0.005674,0.009166,0.000873,0.000000,0.0,0.000000


In [7]:
setup_params = {
    'team_home':team_home,
    'team_away':team_away,
    'minutes': 90,
    'ticks_per_minute': 15,
    'rolling_params' : {
        'keep': 0.1,
        'defence_initiation': 0.07,
        'pass': 0.78,
        'shoot':0.02,
        'dribble': 0.05
    }
}

In [10]:
class TeamPredictors:
    def __init__(self, id=None, data=None):
        self.source_data = data.loc[data['current_team_id'] == id]
        self.team_id = id
        self.home_or_away = self.is_home()

        self.initialize_stats()
    
    def is_home(self):
        if self.team_id == setup_params['team_home']:
            return 'home'
        elif self.team_id == setup_params['team_away']:
            return 'away'

    def goalie_stats(self):
        self.goalie_save_rate = float(self.source_data['save_percentage'].loc[self.source_data['position']== 'Goalkeeper'])
        self.goalie_pass_success = float(self.source_data['pass_accuracy'].loc[self.source_data['position']== 'Goalkeeper'])
        
    def passing_stats(self):
        self.passing_accuracy = float(self.source_data['pass_accuracy'].loc[self.source_data['position'] != 'Goalkeeper'].mean())
        self.key_pass_likelihood = float(self.source_data['key_pass_percentage'].loc[self.source_data['position'] != 'Goalkeeper'].mean())

    def shooting_stats(self):
        self.shooting_accuracy = float(self.source_data['shots_on_rate'].loc[self.source_data['position'] != 'Goalkeeper'].mean())

    def dribble_stats(self):
        self.dribble_success = float(self.source_data['dribble_success_rate'].loc[self.source_data['position'] != 'Goalkeeper'].mean())

    def defensive_stats(self):
        pass

    def initialize_stats(self):
        self.goalie_stats()
        self.passing_stats()
        self.shooting_stats()
        self.dribble_stats()

In [23]:
new_df = df.loc[df['current_team_id']==team_home]
goalie_save_rate = new_df.loc[new_df['position'] =='Goalkeeper']
df.head(25)


,player_id,current_team_id,minutes,position,shots_per_minute,shots_on_rate,goals_per_shot_on,dribbles_per_minute,dribble_success_rate,passes_per_minute,pass_accuracy,key_pass_percentage,assist_rate_per_key_pass,defensive_moves_per_minute,fouls_committed_per_minute,fouls_drawn_per_minute,cards_yellow_per_minute,cards_red_per_minute,penalties_committed_per_minute,save_percentage
0,174,33,1160,Midfielder,0.006897,0.500000,0.250000,0.005172,0.500000,0.650000,0.840849,0.042587,0.074074,0.024138,0.006034,0.002586,0.000862,0.000000,0.0,0.000000
1,526,33,3420,Goalkeeper,0.000000,0.000000,0.000000,0.000292,1.000000,0.401462,0.703569,0.003106,0.000000,0.000585,0.000292,0.001170,0.001462,0.000000,0.0,0.719807
2,742,33,1377,Defender,0.004357,0.333333,0.500000,0.001452,0.500000,0.562092,0.892765,0.001447,0.000000,0.037037,0.005810,0.002179,0.001452,0.000000,0.0,0.000000
3,886,33,3174,Defender,0.006301,0.350000,0.285714,0.015438,0.632653,0.470384,0.837240,0.030400,0.078947,0.042218,0.014178,0.004411,0.001890,0.000315,0.0,0.000000
4,909,33,2291,Attacker,0.017023,0.538462,0.333333,0.044086,0.475248,0.298560,0.792398,0.046125,0.080000,0.010476,0.005674,0.009166,0.000873,0.000000,0.0,0.000000
5,1485,33,3119,Midfielder,0.021802,0.602941,0.243902,0.012825,0.475000,0.613017,0.793410,0.076467,0.068966,0.032382,0.013145,0.008977,0.002886,0.000000,0.0,0.000000
6,2468,65,809,Defender,0.001236,0.000000,0.000000,0.001236,1.000000,0.393078,0.789308,0.035857,0.222222,0.042027,0.007417,0.001236,0.006180,0.000000,0.0,0.000000
7,2771,65,1719,Defender,0.003490,0.166667,1.000000,0.023851,0.658537,0.367656,0.799051,0.021782,0.090909,0.039558,0.010471,0.014543,0.001745,0.000000,0.0,0.000000
8,6056,65,1518,Midfielder,0.009223,0.428571,0.333333,0.009881,0.466667,0.546113,0.854041,0.014124,0.200000,0.056653,0.017128,0.010540,0.002635,0.000000,0.0,0.000000
9,9971,33,1347,Attacker,0.019302,0.615385,0.062500,0.043801,0.440678,0.416481,0.800357,0.048998,0.045455,0.039347,0.014105,0.023756,0.003712,0.000000,0.0,0.000000


In [11]:
team_a = TeamPredictors(team_home, df)
team_b = TeamPredictors(team_away, df)

TypeError: cannot convert the series to <class 'float'>

In [ ]:
class GameManager:
    def __init__(self, team_home, team_away, setup):
        self.team_home = team_home.team_id
        self.team_away = team_away.team_id
        self.posessor = team_home
        self.non_pos = team_away
        self.ball_lost = None
        self.setup = setup

        self.game_log = []

    def update_log(self, type=None):
        if type == 'goal':
            self.game_log.append({
                'team' : self.posessor.team_id,
                'event_type' : 'goal'
            })
        if type == 'pass':
            self.game_log.append({
                'team' : self.posessor.team_id,
                'event_type' : 'pass'
            })
        if type == 'shot':
            self.game_log.append({
                'team' : self.posessor.team_id,
                'event_type' : 'shot'
            })
        if type == 'dribble':
            self.game_log.append({
                'team' : self.posessor.team_id,
                'event_type' : 'dribble'
            })

    def keep_ball(self):
        pass

    def posession_change(self):
        self.ball_lost = self.posessor
        self.posessor = self.non_pos
        self.non_pos = self.ball_lost

    def shoot_ball(self):
        shot_roll = random.random()

        if shot_roll <= self.posessor.shooting_accuracy:
            self.update_log('shot')
            goalie_roll = random.random()

            if goalie_roll >= self.non_pos.goalie_save_rate:
                self.update_log('goal')
                self.posession_change()
            else:
                self.posession_change()

    def pass_ball(self):
        pass_roll = random.random()

        if pass_roll <= self.posessor.passing_accuracy:
            self.update_log('pass')
            key_pass_roll = random.random()
            if key_pass_roll <= self.posessor.key_pass_likelihood:
                self.shoot_ball()
            else:
                pass
        else:
            self.posession_change()

    def dribble_try(self):
        dribble_roll = random.random()
        
        if dribble_roll <=self.posessor.dribble_success:
            self.update_log('dribble')
            pass
        else:
            self.posession_change()

    def spin_wheel(self, roll_params):
        keep = roll_params['keep']
        defence_init = keep + roll_params['defence_initiation']
        pass_init = defence_init + roll_params['pass']
        shoot_init = pass_init + roll_params['shoot']
        dribble_init = shoot_init + roll_params['dribble']

        dice_roll = random.random()

        if dice_roll <= keep:
            self.keep_ball()
        elif keep < dice_roll <= defence_init:
            self.posession_change()
        elif defence_init < dice_roll <= pass_init:
            self.pass_ball()
        elif pass_init < dice_roll <= shoot_init:
            self.shoot_ball()
        elif shoot_init < dice_roll:
            self.dribble_try()

    def generate_score(self, print_or_no = False):
        if len(self.game_log) == 0:
            if print_or_no == True:
                print(f'Team {self.team_home}: 0 || Team {self.team_away}: 0')
            else:
                return (0,0)
        else:
            home_score  = 0
            away_score = 0
            for event in self.game_log:
                if event['team'] == self.team_home:
                    home_score += 1
                elif event['team'] == self.team_away:
                    away_score += 1
            if print_or_no == True:
                print(f'Team {self.team_home}: {home_score} || Team {self.team_away}: {away_score}')
            else:
                return (home_score, away_score)
    
    def log_results(self):
        home_goal  = 0
        home_pass = 0
        home_shot = 0
        home_dribble = 0
        away_goal = 0
        away_shot = 0
        away_pass = 0
        away_dribble = 0
        for event in self.game_log:
            if event['event_type'] == 'goal':
                if event['team'] == self.team_home:
                    home_goal += 1
                else:
                    away_goal += 1
            if event['event_type'] == 'shot':
                if event['team'] == self.team_home:
                    home_shot += 1
                else:
                    away_shot += 1

            if event['event_type'] == 'pass':
                if event['team'] == self.team_home:
                    home_pass += 1
                else:
                    away_pass += 1

            if event['event_type'] == 'dribble':
                if event['team'] == self.team_home:
                    home_dribble += 1
                else:
                    away_dribble += 1

        return (home_goal, away_goal, home_shot, away_shot, home_pass, away_pass, home_dribble, away_dribble)

    def initialize_game(self):
        for segment in range(self.setup['minutes'] * self.setup['ticks_per_minute']):
            self.spin_wheel(self.setup['rolling_params'])
        # return self.generate_score()
        return self.log_results()

In [ ]:
iterations = 10
scores_df = pd.DataFrame(columns=['home_goal', 'away_goal', 'home_shot', 'away_shot', 'home_pass', 'away_pass', 'home_dribble', 'away_dribble'])
for i in range(iterations):
    test = GameManager(team_a, team_b, setup_params)
    scores_df.loc[len(scores_df)] = test.initialize_game()
output = scores_df.describe().loc['mean']

In [ ]:
print(output)